In [ ]:
import numpy as np
import qutip as qt
import pandas as pd

from pulsee import simulation as sim, operators as op, spin_squeezing as sqz
import pulsee.plot as psplt

In [ ]:
QUANTUM_NUMBERS = [3/2] # spin 3/2
GAMMA_2PIS = [4.17169] # = gamma / 2pi

# Place the nulceus in a field
B0 = 1
W0 = GAMMA_2PIS[0] * B0 * 2 * np.pi 
B1 = 1e-10
zeem_par = {'field magnitude' : B0, 'theta_z' : 0, 'phi_z' : 0}

ACQUISITION_TIME = 800 # in microseconds
T2 = 500

h_dict = {
}

args = h_dict.copy()
spin_par = []
for qn, gam in zip(QUANTUM_NUMBERS, GAMMA_2PIS):
    spin_par.append({'quantum number': qn, 'gamma/2pi': gam})

WQ = 1 * 2 * np.pi
quad_par = [{'coupling constant' : 3 * WQ/np.pi,
                    'asymmetry parameter' : 0,
                    'alpha_q' : np.pi,
                    'beta_q' : 0.,
                    'gamma_q' : 0.,
                    'order' : 0}]

args['spin_par'] = spin_par 
args['zeem_par'] = zeem_par
args['quad_par'] = quad_par

In [ ]:
pulse_time = 2 / (4 * 3 * GAMMA_2PIS[0] * B1)
mode = pd.DataFrame([( B0 * GAMMA_2PIS[0], B1, 0., np.pi/2, 0, pulse_time)], 
                           columns=['frequency', 'amplitude', 'phase', 'theta_p', 'phi_p', 'pulse_time'])

In [ ]:
initial_state={'theta' : np.deg2rad(90), 'phi' : np.deg2rad(180)}

In [ ]:
spin, _, dm_initial = sim.nuclear_system_setup(spin_par, quad_par=quad_par,
                                            zeem_par=zeem_par, initial_state=[{'theta' : np.deg2rad(90), 'phi' : np.deg2rad(180)}])
sqz_ops = sqz.useful_sqz_ops(spin)
Ix, Iy, Iz, Ix2, Iy2, Iz2, Ip2, Ip_2Iz = (sqz_ops.Ix, sqz_ops.Iy, sqz_ops.Iz, sqz_ops.Ix2, sqz_ops.Iy2, sqz_ops.Iz2, sqz_ops.Ip2, sqz_ops.Ip_2Iz)

In [ ]:
def defHlab(eta, omegaQ, gammaS, field=B0):
    Hamil = -gammaS*field*Iz
    
    Hamil += omegaQ*((3*Iz2)/2)
    #omegaQ is (uni)axail anisotropy?

    Hamil += omegaQ*eta*(Ix2 - Iy2)/2
    #omegaT transverse anisotropy
    #eta = omegaT/omegaQ, let's keep omegaQ constant
    return [Hamil]

In [ ]:
h_unperturbed = defHlab(0, WQ, 0, 0)

In [ ]:
# sqz.CSS(spin, initial_state={'theta' : np.deg2rad(9), 'phi' : np.deg2rad(180)})

In [ ]:
fig = psplt.plot_complex_density_matrix(dm_initial)

In [1]:
wQaxis = 1/(3*WQ)
times = np.arange(0, wQaxis, 1e-3)
# times=

NameError: name 'WQ' is not defined

In [ ]:
h_unperturbed

In [ ]:
opts = qt.Options(atol=1e-16, rtol=1e-16, rhs_reuse=False, nsteps=40000)
dms = sim.evolve(spin, h_unperturbed, dm_initial, solver='mesolve', \
                        mode=mode, times=times, opts=opts, return_allstates=True)

In [ ]:
sqz_ops = sqz.populate_averge_values(dms, sqz_ops)

In [ ]:
It = np.sqrt(sqz_ops.avIx**2+sqz_ops.avIy**2+sqz_ops.avIy**2)
sqz.plot_values([sqz_ops.avIx, sqz_ops.avIy, sqz_ops.avIz, It], times, [2,2],
                wQaxis, x_label=r'Scaled Time ($\omega_Q^{-1}$)')

In [ ]:
DIx = np.sqrt(sqz_ops.avIx2-sqz_ops.avIx**2)
DIy = np.sqrt(sqz_ops.avIy2-sqz_ops.avIy**2)
DIz = np.sqrt(sqz_ops.avIz2-sqz_ops.avIz**2)

sqz.plot_values([DIx, DIy, DIz], times, 1, wQaxis, title='Standard deviation values', 
                y_label='Standard Deviation', x_label='Scaled Time ($\omega_Q^{-1}$)', labels=['\Delta I_x', '\Delta I_y', '\Delta I_z'])

In [ ]:
xi, alpha, Jn_1, Jn_2, Jn_3 = sqz.calc_squeez_param(sqz_ops, spin.I['I'], xi_sq=True, return_av_spher=True)
sqz.plot_values([xi, alpha], times, [2, 1], wQaxis, title=r'Gerenalized $\xi^2$',
                labels=["\\xi^2", "\\alpha_\\xi"], x_label='Scaled Time ($\omega_Q^{-1}$)', put_brackets=False)

In [ ]:
sqz.plot_values([Jn_1, Jn_2, Jn_3], times, 1, wQaxis, title='Mean values of magnetization in spherical',
    x_label='Scaled Time ($\omega_Q^{-1}$)', labels=['Jn_1', 'Jn_2', 'Jn_3'],put_brackets=True)